In [ ]:
!pip install fastai==0.7.0

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.imports import *
from fastai.structured import *

from sklearn.ensemble import RandomForestRegressor
from IPython.display import display

from sklearn import metrics

In [ ]:
PATH = '../input'
!ls {PATH}

In [ ]:
types = { 'id': 'int64',
          'item_nbr': 'int32',
          'store_nbr': 'int8',
          'unit_sales': 'float32',
          'onpromotion': 'object'}

In [ ]:
%%time
df_all = pd.read_csv(f'{PATH}/train.csv', parse_dates=['date'], dtype=types,
                     infer_datetime_format = True)

In [ ]:
df_all.onpromotion.fillna(False, inplace = True)
df_all.onpromotion = df_all.onpromotion.map({'False': False, 'True': True})
df_all.onpromotion = df_all.onpromotion.astype(bool)

!mkdir tmp
%time df_all.to_feather('tmp/raw_groceries')

In [ ]:
%time df_all.describe(include='all')

In [ ]:
df_all.tail()

In [ ]:
df_all = pd.read_feather('tmp/raw_groceries')

In [ ]:
df_all.unit_sales = np.log1p(np.clip(df_all.unit_sales, 0, None))

In [ ]:
def add_datepart_large(temp_df, size):
    list_df = [temp_df[i:i+size] for i in range(0,temp_df.shape[0],size)]
    for i in range(len(list_df)): add_datepart(list_df[i], 'date')
    return pd.concat(list_df)

final_df = add_datepart_large(df_all, 10000)